In [ ]:
import time
import numpy as np
import random as rd
import math
import os
import torch
from pathlib import Path
import glob
from torch.utils.data import Dataset,DataLoader,random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint,EarlyStopping,Callback
from torchmetrics import MeanSquaredError
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.trial import TrialState

In [ ]:
"""
random
u_min=1
u_max=100
=> (1,100)
linear
u_min=30
u_max=1200
=> (3,120)
quadratic
u_min=300
u_max=120000
=> (1,400)


max demand in the 4-th dimension can be sqrt(10000+10000+10000)=100*sqrt(3)<=174,min=sqrt(3)>1

Total demand per dimension can be max 96*100=9600,so 0.7 part of it is 6720 = max capacity

max_utility=400

overall (1->400)
"""


def problem_grouping(clients_array, time_slot, quad_constr, util_rate=1.5,
                     time_len=16, num_of_util_groups=15,
                     num_of_demand_groups=16,
                     max_quadratic_demand=174,
                     max_utility=400):  ##grouping by utility,then demand,then by time_slot_length
    clients = clients_array.copy()
    num_of_dims = time_slot.shape[0]
    # Group by utilities
    max_util = clients[:, -2].max()
    if 100 < max_util <= 1200:
        clients[:, -2] /= 10
    elif max_util > 1200:
        clients[:, -2] /= 300
    util_groups = []
    steps = np.zeros(num_of_util_groups)
    steps[0] = util_rate
    for i in range(1, steps.size):
        steps[i] = steps[i - 1] * util_rate
    bool_idx = (clients[:, -2] <= steps[0])
    util_groups.append(clients[np.where(bool_idx)[0]])
    for i in range(1, num_of_util_groups):
        bool_idx = (clients[:, -2] <= steps[i]) & (
                clients[:, -2] > steps[i - 1])
        util_groups.append(clients[np.where(bool_idx)[0]])
    # Normalizing by max_utility
    clients /= max_utility
    max_util = clients[:, -2].max()
    min_util = clients[:, -2].min()
    total_utility = clients[:, -2].sum()
    for group in util_groups:
        if group.size > 0:
            group[:, -2] /= max_utility
    # Group by demand of 4th - quadratic dimension
    demand_groups = []
    step = max_quadratic_demand / num_of_demand_groups
    demand_steps = np.arange(0, max_quadratic_demand + 1, step)
    for i in range(num_of_util_groups):
        if util_groups[i].size > 0:
            group_demands = [np.sqrt((client[:num_of_dims] ** 2).sum()) for client in util_groups[i]]
            for j in range(num_of_demand_groups):
                bool_idx = (group_demands > demand_steps[j]) & (group_demands <= demand_steps[j + 1])
                demand_groups.append(util_groups[i][np.where(bool_idx)[0]])
        else:
            for _ in range(num_of_demand_groups):
                demand_groups.append(util_groups[i])

    # Normalizing demands in each dimension by max capacity
    max_cap = time_slot.max()
    for group in demand_groups:
        group[:, num_of_dims] /= max_cap
    # sum of demands of the first dimension normalized by max capacity
    total_demand = 0
    for group in demand_groups:
        total_demand += group[:, 0].sum()
    # Normalizing capacities by max capacity
    time_slot_normalized = time_slot / max_cap
    time_slot_min = time_slot_normalized.min(axis=1)
    time_slot_av = time_slot_normalized.mean(axis=1)
    time_slot_max = time_slot_normalized.max(axis=1)
    min_av_max_caps = np.concatenate((time_slot_min, time_slot_av, time_slot_max))
    quad_constr_normalized = quad_constr / max_cap
    av_cap_bytime = time_slot.mean(axis=0) / max_cap
    # Group by time_length
    final_groups = []
    time_horizon = time_slot.shape[1]
    num_of_time_groups = int(time_horizon / time_len)
    time_steps = np.arange(0, time_horizon + 1, time_len)
    for i in range(num_of_demand_groups * num_of_util_groups):
        if demand_groups[i].size > 0:
            group_time_lengths = demand_groups[i][:, num_of_dims + 1] - demand_groups[i][:, num_of_dims]
            for j in range(num_of_time_groups):
                bool_idx = (group_time_lengths > time_steps[j]) & (group_time_lengths <= time_steps[j + 1])
                final_groups.append(demand_groups[i][np.where(bool_idx)[0]])
        else:
            for _ in range(num_of_time_groups):
                final_groups.append(demand_groups[i])

    return final_groups, total_demand, min_av_max_caps, av_cap_bytime, quad_constr_normalized, min_util, max_util, total_utility


def data_preprocess(groups, total_demand, quad_cap, min_av_max_caps, av_cap_bytime,
                    min_util, max_util, total_utility, num_of_dims=3, num_of_clients=1500, time_length=96):
    length = len(groups)
    inputs = np.zeros((length + 1, 3 * num_of_dims + 1 + time_length + 2))
    labels = np.zeros((length + 1, 3))
    for i, group in enumerate(groups):
        if group.size > 0:
            inputs[i][:num_of_dims] = np.min(group[:, :num_of_dims], axis=0)  # minimum demand per dimension
            inputs[i][num_of_dims:2 * num_of_dims] = np.mean(group[:, :num_of_dims],
                                                             axis=0)  # average demand per dimension
            inputs[i][2 * num_of_dims:3 * num_of_dims] = np.max(group[:, :num_of_dims],
                                                                axis=0)  # maximum demand per dimension
            quadr_demands = ((group[:, :num_of_dims] ** 2).sum(axis=1)).mean()  # mean demand in the quadratic dimension
            inputs[i][3 * num_of_dims] = quadr_demands
            inputs[i][3 * num_of_dims + 1:3 * num_of_dims + time_length + 1] = np.sum(
                group[:, num_of_dims + 2:num_of_dims + 2 + time_length],
                axis=0) / num_of_clients  # average time_occupancy

            inputs[i][-2] = np.mean(group[:, -2])  # average utility
            inputs[i][-1] = group.shape[0] / num_of_clients  # what part of clients is in this group
            labels[i][0] = np.sum(group[:, -1]) / num_of_clients  # what part is selected in the final answer
            if group[np.where(group[:, -1] == 1)].size > 0:
                # total demand of selected clients/total demand of the first dimension
                selected = group[np.where(group[:, -1] == 1)]
                labels[i][1] = selected[:, 0].sum() / total_demand
                labels[i][2] = selected[:, -2].sum() / total_utility
    inputs[-1][:3 * num_of_dims] = min_av_max_caps
    inputs[-1][3 * num_of_dims] = quad_cap
    inputs[-1][3 * num_of_dims + 1:3 * num_of_dims + time_length + 1] = av_cap_bytime
    inputs[-1][-2] = min_util
    inputs[-1][-1] = max_util
    labels[-1][0] = 1 - labels[:, 0].sum()
    labels[-1][1] = 1 - labels[:, 1].sum()
    labels[-1][2] = 1 - labels[:, 2].sum()
    return inputs, labels

In [ ]:
class DataSetMaker(Dataset):
    def __init__(self):
        self.states=list(glob.glob('states/*.npz'))
    def __len__(self):
        return len(self.states)

    def __getitem__(self, idx):
        state = np.load(self.states[idx])
        clients_list = state['cl']
        time_slot_capacity = state['tslot']
        quad_constr = state['quad_constr']
        f_groups, tot_dem, mi_av_max_caps, av_cap_time, q_c, mi_util, ma_util, tot_util = problem_grouping(
            clients_list,
            time_slot_capacity,
            quad_constr)
        inp, lab = data_preprocess(f_groups, tot_dem, q_c, mi_av_max_caps, av_cap_time,
                                   mi_util, ma_util, tot_util) 
        return np.float32(inp), np.float32(lab)

In [ ]:
dataset=DataSetMaker()

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

In [ ]:
train_set,valid_set,test_set=random_split(dataset,[7439,1000,1000])

In [ ]:
train_loader = DataLoader(train_set, batch_size=8,shuffle=True,drop_last=False)
val_loader = DataLoader(valid_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

In [ ]:
def scaled_dot_product(q, k, v):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values

In [ ]:
class MultiheadAttention(nn.Module):

    def __init__(self, input_dim, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Stack all weight matrices 1...h together for efficiency
        # Note that in many implementations you see "bias=False" which is optional
        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, embed_dim)

        self._reset_parameters()

    def _reset_parameters(self):
        # Original Transformer initialization, see PyTorch documentation
        nn.init.xavier_uniform_(self.qkv_proj.weight)
        self.qkv_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.o_proj.weight)
        self.o_proj.bias.data.fill_(0)

    def forward(self, x):
        batch_size, seq_length, _ = x.size()
        qkv = self.qkv_proj(x)

        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)

        # Determine value outputs
        values = scaled_dot_product(q, k, v)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        values = values.reshape(batch_size, seq_length, self.embed_dim)
        o = self.o_proj(values)

        
        return o

In [ ]:
class EncoderBlock(nn.Module):

    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()

        # Attention layer
        self.self_attn = MultiheadAttention(input_dim, input_dim, num_heads)

        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )

        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Attention part
        attn_out = self.self_attn(x)
        x = x + self.dropout(attn_out)
        x = self.norm1(x)

        # MLP part
        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm2(x)

        return x

class TransformerEncoder(nn.Module):

    def __init__(self, num_layers, **block_args):
        super().__init__()
        self.layers = nn.ModuleList([EncoderBlock(**block_args) for _ in range(num_layers)])

    def forward(self, x):
        for l in self.layers:
            x = l(x)
        return x

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model)
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [ ]:
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [ ]:
class TransformerPredictor(pl.LightningModule):

    def __init__(self, input_dim, model_dim, num_classes, num_heads, num_layers, lr, warmup, max_iters, dropout=0.0, input_dropout=0.0):
        """
        Inputs:
            input_dim - Hidden dimensionality of the input
            model_dim - Hidden dimensionality to use inside the Transformer
            num_classes - Number of classes to predict per sequence element
            num_heads - Number of heads to use in the Multi-Head Attention blocks
            num_layers - Number of encoder blocks to use.
            lr - Learning rate in the optimizer
            warmup - Number of warmup steps. Usually between 50 and 500
            max_iters - Number of maximum iterations the model is trained for. This is needed for the CosineWarmup scheduler
            dropout - Dropout to apply inside the model
            input_dropout - Dropout to apply on the input features
        """
        super().__init__()
        self.save_hyperparameters()
        self._create_model()

    def _create_model(self):
        # Input dim -> Model dim
        self.input_net = nn.Sequential(
            nn.Dropout(self.hparams.input_dropout),
            nn.Linear(self.hparams.input_dim, self.hparams.model_dim)
        )
        # Positional encoding for sequences
        self.positional_encoding = PositionalEncoding(d_model=self.hparams.model_dim)
        # Transformer
        self.transformer = TransformerEncoder(num_layers=self.hparams.num_layers,
                                              input_dim=self.hparams.model_dim,
                                              dim_feedforward=2*self.hparams.model_dim,
                                              num_heads=self.hparams.num_heads,
                                              dropout=self.hparams.dropout)
        # Output classifier per sequence lement
        self.output_net = nn.Sequential(
            nn.Linear(self.hparams.model_dim, self.hparams.model_dim),
            nn.LayerNorm(self.hparams.model_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(self.hparams.dropout),
            nn.Linear(self.hparams.model_dim, self.hparams.num_classes),
            nn.ReLU(inplace=True)
        )

    def forward(self, x,add_positional_encoding=False):
        """
        Inputs:
            x - Input features of shape [Batch, SeqLen, input_dim]
            mask - Mask to apply on the attention outputs (optional)
            add_positional_encoding - If True, we add the positional encoding to the input.
                                      Might not be desired for some tasks.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        x = self.transformer(x)
        x = self.output_net(x)
        x = F.softmax(x, dim=-2)
        return x

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)

        lr_scheduler = CosineWarmupScheduler(optimizer,
                                             warmup=self.hparams.warmup,
                                             max_iters=self.hparams.max_iters)
        
        return [optimizer], [{'scheduler': lr_scheduler,
                              'interval': 'epoch'}]
#         optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)
#         self.lr_scheduler = CosineWarmupScheduler(
#             optimizer, warmup=self.hparams.warmup, max_iters=self.hparams.max_iters
#         )
#         return optimizer
    
#     def optimizer_step(self, *args, **kwargs):
#         super().optimizer_step(*args, **kwargs)
#         self.lr_scheduler.step()  # Step per iteration
        
    def training_step(self, batch, batch_idx):
        raise NotImplementedError

    def validation_step(self, batch, batch_idx):
        raise NotImplementedError

    def test_step(self, batch, batch_idx):
        raise NotImplementedError
        
    def on_after_backward(self):
        raise NotImplementedError

In [ ]:
class  KnapsackPredictor(TransformerPredictor):

    def _calculate_loss(self, batch, mode="train"):
        inp_data, labels = batch
        batch_size = labels.shape[0]
        preds = self.forward(inp_data)
        loss1 = (1 / batch_size) * (torch.mean(torch.abs(preds[:, :-1, 0] - labels[:, :-1, 0])) + 1 / 720 * torch.mean(
            torch.abs(preds[:, -1, 0] - labels[:, -1, 0])))
        loss2 = 1 / batch_size * (torch.mean(torch.abs(preds[:, :-1, 1] - labels[:, :-1, 1])) + 1 / 720 * torch.mean(
            torch.abs(preds[:, -1, 1] - labels[:, -1, 1])))
        loss3 = 1 / batch_size * (torch.mean(torch.abs(preds[:, :-1, 2] - labels[:, :-1, 2])) + 1 / 720 * torch.mean(
            torch.abs(preds[:, -1, 2] - labels[:, -1, 2])))
        loss = loss1 + loss2 + loss3

        mse_error_1 = F.mse_loss(preds[:, :, 0], labels[:, :, 0])
        mse_error_2 = F.mse_loss(preds[:, :, 1], labels[:, :, 1])
        mse_error_3 = F.mse_loss(preds[:, :, 2], labels[:, :, 2])
        
        
        non_zero_labels=labels[:, :, 0]!=0
        relative_accuracy_1 = (abs(preds[non_zero_labels][:,0] - labels[non_zero_labels][:,0]) < 0.1 * labels[non_zero_labels][:,0]).sum()
        relative_accuracy_1 += (preds[~non_zero_labels][:,0]<(2/1500)).sum()
        relative_accuracy_1 = relative_accuracy_1/(preds.shape[0]*preds.shape[1])
        
        relative_accuracy_2 = (abs(preds[non_zero_labels][:,1] - labels[non_zero_labels][:,1]) < 0.1 * labels[non_zero_labels][:,1]).sum()
        relative_accuracy_2 += (preds[~non_zero_labels][:,1]<1e-4).sum()
        relative_accuracy_2 = relative_accuracy_2/(preds.shape[0]*preds.shape[1])
        
        relative_accuracy_3 = (abs(preds[non_zero_labels][:,2] - labels[non_zero_labels][:,2]) < 0.1 * labels[non_zero_labels][:,2]).sum()
        relative_accuracy_3 += (preds[~non_zero_labels][:,2]<1e-4).sum()
        relative_accuracy_3 = relative_accuracy_3/(preds.shape[0]*preds.shape[1])

        #         Logging
        self.log(f"{mode}_loss", loss)
        self.log(f"{mode}_loss1", loss1)
        self.log(f"{mode}_loss2", loss2)
        self.log(f"{mode}_loss3", loss2)
        self.log(f"{mode}_mse_1", mse_error_1)
        self.log(f"{mode}_mse_2", mse_error_2)
        self.log(f"{mode}_mse_3", mse_error_3)
        self.log(f"{mode}_relative_accuracy_1", relative_accuracy_1)
        self.log(f"{mode}_relative_accuracy_2", relative_accuracy_2)
        self.log(f"{mode}_relative_accuracy_3", relative_accuracy_3)

        return loss
    
    
    def training_step(self, batch, batch_idx):
        loss= self._calculate_loss(batch, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")
    
    def on_after_backward(self):
        if self.trainer.global_step % self.trainer.log_every_n_steps == 0: 
            for name, param in self.named_parameters():
                if param.requires_grad:
                    self.logger.experiment.add_scalar(f"grad_norm/{name}", param.grad.norm(), self.global_step)

In [ ]:
CHECKPOINT_PATH="C:\\Users\\zhira\\CSIE_PYTHON_PROJECTS\\UFP_FINAL"

In [ ]:
class WeightLoggingCallback(Callback):
    def on_epoch_end(self, trainer, pl_module):
        for name, param in pl_module.named_parameters():
            # Calculate statistics
            mean_val = param.mean()
            std_val = param.std()
            max_val = param.max()
            min_val = param.min()

            # Log the statistics
            trainer.logger.experiment.add_scalar(f'{name}_mean', mean_val, trainer.current_epoch)
            trainer.logger.experiment.add_scalar(f'{name}_std', std_val, trainer.current_epoch)
            trainer.logger.experiment.add_scalar(f'{name}_max', max_val, trainer.current_epoch)
            trainer.logger.experiment.add_scalar(f'{name}_min', min_val, trainer.current_epoch)

In [ ]:
def train_knapsack(**kwargs):
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "UfpCheckPoint")
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(callbacks=[ModelCheckpoint(dirpath=root_dir,filename='UfpCheckPointNew2',save_weights_only=True, mode="min", monitor="val_loss"),
                                    LearningRateMonitor(logging_interval='epoch'),
                                    EarlyStopping(monitor="val_loss", min_delta=1e-7, patience=20, verbose=False, mode="min"),WeightLoggingCallback()],
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=150,
                         log_every_n_steps=10)
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(root_dir, "UfpCheckPoint.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = KnapsackPredictor.load_from_checkpoint(pretrained_filename)
    else:
        model = KnapsackPredictor(max_iters=trainer.max_epochs, **kwargs)
        trainer.fit(model, train_loader, val_loader)

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=1)
    test_result = trainer.test(model, test_loader, verbose=1)
    result = {"test_acc": test_result, "val_acc": val_result}

    model = model.to(device)
    return model,result

In [ ]:
knapsack_model,result =                  train_knapsack(input_dim=108,
                                                 model_dim=156,
                                                 num_heads=2,
                                                 num_classes=3,
                                                 num_layers=2,
                                                 dropout=0.0,
                                                 lr=2.8e-4,
                                                 warmup=20)

In [ ]:
result

In [ ]:
def objective(trial: optuna.trial.Trial):
    # Define the hyperparameters to optimize
    model_dim = trial.suggest_int('model_dim', 108, 204, 12)
    num_heads = trial.suggest_int('num_heads', 2, 6, 2)
    num_layers = trial.suggest_int('num_layers', 2, 4)
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    dropout = trial.suggest_float('dropout', 0, 0.2, step=0.1)

    # Create a model instance with the suggested hyperparameters
    model = KnapsackPredictor(108, model_dim, 3, num_heads, num_layers, lr, dropout, 0.0)

    # Create a PyTorch Lightning trainer with the Optuna Pruner
    trainer = pl.Trainer(
        callbacks=[PyTorchLightningPruningCallback(trial, monitor='val_loss')],
        max_epochs=20,
        accelerator="gpu" if str(device).startswith("cuda") else "cpu",
        devices=1
    )
    trainer.fit(model, train_loader, val_loader)

    val_loss = trainer.callback_metrics["val_loss"].item()

    return val_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters
best_trial = study.best_trial
print(f"Best trial: {best_trial.params}")

best_trial_dict = best_trial.params
best_trial_dict["value"] = best_trial.value

with open('best_trial.json', 'w') as outfile:
    json.dump(best_trial_dict, outfile, indent=4)

In [ ]:
inps,labs=test_set.__getitem__(0)

In [ ]:
inps=torch.from_numpy(np.float32(inps[None,:])).to(device)

In [ ]:
pred=knapsack_model(inps)

In [ ]:
pred.shape

In [ ]:
pred=pred.cpu().detach().numpy().squeeze()

In [ ]:
labs[labs[:,0]!=0][:-1,2].min()

In [ ]:
labs[labs[:,0]!=0][:-1].mean()

In [ ]:
labs.shape

In [ ]:
plt.scatter(pred[:-1,0],labs[:-1,0])
plt.plot([0,0.03],[0,0.03])

In [ ]:
plt.scatter(pred[:-1,1],labs[:-1,1])
plt.plot([0,0.04],[0,0.04])

In [ ]:
plt.scatter(pred[:-1,2],labs[:-1,2])
plt.plot([0,0.04],[0,0.04])

In [ ]:
def selection_algorithm(clients_list,time_slot,groups,total_demand,y_pred):
    num_of_changes=10
    num_of_dims=time_slot.shape[0]
    if type(y_pred)!=np.ndarray:
        prediction=y_pred.cpu().detach().numpy()
    else:
        prediction=y_pred
    length=clients_list.shape[0]
    time_length=time_slot.shape[1]
    time_slot_min=np.zeros(num_of_dims)
    for dim in range(num_of_dims):
        time_slot_min[dim]=time_slot[dim].min()
    sum_demands=prediction[:,1]*total_demand #(total_demand)<-total demand of selected items in the first dimension
    violated_total_utility=0
    violated_selected_items=[]
    final_total_utility=0
    final_selected_items=[]
    for step in range(100):
        total_utility=0
        selected_items=[]
        for i,group in enumerate(groups):
            if(group.size>0):
                max_res=0
                best_vec=np.array([])
                curr_demand=0
                group_util_sum=np.sum(group[:,-2])
                #prob_vec=np.array([item[-2]/group_util_sum for item in group])
                prob_vec_uniform=np.ones(group.shape[0])/group.shape[0]
                for _ in range(10):
                    sel_size=min(math.ceil(prediction[i][0]*length),group.shape[0])
                    selection=np.random.choice(range(group.shape[0]),sel_size,replace=False,p=prob_vec_uniform)
                    sel_vec=group[selection]
                    util_sum=np.sum(sel_vec[:,-2])
                    d_sum=np.sum(sel_vec[:,0]) #total demand of selected items in the first dimension
                    if np.abs(d_sum-sum_demands[i])<=0.1*sum_demands[i] and util_sum>max_res:
                        max_res=util_sum
                        best_vec=sel_vec
                        curr_demand=d_sum
                if best_vec.size>0:
                    total_utility+=max_res
                    for item in best_vec:
                        selected_items.append(item)
        is_correct=np.stack([sum([selected_items[index][dim]*time_slot_min[dim]*selected_items[index][num_of_dims+1+time] for index in range(len(selected_items))])<=time_slot[dim][time-1]
                                                  for dim in range(num_of_dims) for time in range(1,time_length+1)])
        if all(is_correct):
            if final_total_utility<total_utility:
                final_selected_items=selected_items.copy()
                final_total_utility=total_utility
        else:
            if violated_total_utility<total_utility:
                violated_selected_items=selected_items.copy()
                violated_total_utility=total_utility
                correct=np.copy(is_correct)
    if final_total_utility>0:
        return np.stack(final_selected_items),final_total_utility
    while not all(correct):
            min_utility=2
            min_item_index=0
            indices=np.where(correct==False)[0]
            dims_f=indices%6
            times_f=indices//6+1
            time_indices=num_of_dims+1+times_f
            for item in range(len(violated_selected_items)):
                x=violated_selected_items[item]
                if any(x[time_indices]==1):
                    if(x[-2]<min_utility):
                        min_utility=x[-2]
                        min_item_index=item
            violated_selected_items.pop(min_item_index)
            violated_total_utility=sum([item[-2] for item in violated_selected_items])
            correct=np.stack([sum([violated_selected_items[index][dim]*time_slot_min[dim]*violated_selected_items[index][num_of_dims+1+time] for index in range(len(violated_selected_items))])<=time_slot[dim][time-1]
                                              for dim in range(num_of_dims) for time in range(1,time_length+1)])
            if all(correct):
                break
    return np.stack(violated_selected_items),violated_total_utility

In [ ]:
def selection_algorithm2(clients_list,time_slot,groups,total_demand,y_pred):
    if type(y_pred)!=np.ndarray:
        prediction=y_pred.cpu().detach().numpy()
    else:
        prediction=y_pred
    num_of_dims=time_slot.shape[0]
    length=clients_list.shape[0]
    time_length=time_slot.shape[1]
    time_slot_min = time_slot.min(axis=1)
    
    sum_demands=prediction[:,1]*total_demand #(total_demand)<-total demand of selected items in the first dimension
    final_total_utility=0
    final_selected_items=[]
    for step in range(100):
        total_utility=0
        selected_items=[]
        for i,group in enumerate(groups):
            if(group.size>0):
                max_res=0
                best_vec=np.array([])
                curr_demand=0
                group_util_sum=np.sum(group[:,-2])
                #prob_vec=np.array([item[-2]/group_util_sum for item in group])
                prob_vec_uniform=np.ones(group.shape[0])/group.shape[0]
                for _ in range(10):
                    sel_size=min(math.ceil(prediction[i][0]*length),group.shape[0])
                    selection=np.random.choice(range(group.shape[0]),sel_size,replace=False,p=prob_vec_uniform)
                    sel_vec=group[selection]
                    util_sum=np.sum(sel_vec[:,-2])
                    d_sum=np.sum(sel_vec[:,0]) #total demand of selected items in the first dimension
                    if util_sum>max_res and np.abs(d_sum-sum_demands[i])<=0.1*sum_demands[i]:#0.05
                        max_res=util_sum
                        best_vec=sel_vec
                        curr_demand=d_sum
                if best_vec.size>0:
                    total_utility+=max_res
                    selected_items.extend(best_vec)
                    
        is_correct=np.stack([sum([selected_items[index][dim]*time_slot_min[dim]*selected_items[index][num_of_dims+1+time] for index in range(len(selected_items))])<=time_slot[dim][time-1]
                                                  for dim in range(num_of_dims) for time in range(1,time_length+1)])
        while not all(is_correct) and total_utility>final_total_utility:
            min_utility=2
            min_item_index=0
            indices=np.where(is_correct==False)[0]
            dims_f=indices%6
            times_f=indices//6+1
            time_indices=num_of_dims+1+times_f
            for item in range(len(selected_items)):
                x=selected_items[item]
                if any(x[time_indices]==1):
                    if(x[-2]<min_utility):
                        min_utility=x[-2]
                        min_item_index=item
            selected_items.pop(min_item_index)
            total_utility=sum([item[-2] for item in selected_items])
            is_correct=np.stack([sum([selected_items[index][dim]*time_slot_min[dim]*selected_items[index][num_of_dims+1+time] for index in range(len(selected_items))])<=time_slot[dim][time-1]
                                              for dim in range(num_of_dims) for time in range(1,time_length+1)])
            if all(is_correct):
                break
        if final_total_utility<total_utility:
            final_selected_items=selected_items.copy()
            final_total_utility=total_utility
    return np.stack(final_selected_items),final_total_utility

In [ ]:
def selection_algorithm3(clients_list, time_slot, groups, total_demand, y_pred):
    if type(y_pred) != np.ndarray:
        prediction = y_pred.cpu().detach().numpy()
    else:
        prediction = y_pred

    num_of_dims = time_slot.shape[0]
    length = clients_list.shape[0]
    time_length = time_slot.shape[1]
    time_slot_min = time_slot.min(axis=1)
    
    sum_demands = prediction[:, 1] * total_demand 

    final_total_utility = 0
    final_selected_items = []

    for _ in range(10):
        total_utility = 0
        selected_items = []
        rand_perm = np.random.permutation(len(groups))
        
        for gr_num in rand_perm:
            group = groups[gr_num]
            if group.size > 0:
                max_res = 0
                best_vec = np.array([])

                sel_size = min(math.ceil(prediction[gr_num][0] * length), group.shape[0])
                indexes = set(range(sel_size))
                chosen = set()
                invalid_indices = set()

                for _ in range(10):
                    sel_vec = []

                    for index in range(sel_size):
                        for _ in range(10):
                            available_indexes = list(indexes - chosen - invalid_indices)
                            if not available_indexes:
                                break
                            
                            choice = np.random.choice(available_indexes)
                            
                            curr_item = group[choice]
                            temp_selected_items = selected_items + [curr_item]

                            sums = np.array([sum([item[dim] * time_slot_min[dim] * item[num_of_dims + 1 + time] for item in temp_selected_items])
                                             for dim in range(num_of_dims) for time in range(1, time_length + 1)])

                            is_correct = sums <= time_slot[:,:time_length].flatten()

                            if all(is_correct):
                                sel_vec.append(curr_item)
                                chosen.add(choice)
                                break
                            else:
                                invalid_indices.add(choice)

                    if sel_vec:
                        sel_vec = np.array(sel_vec)
                        util_sum = sel_vec[:, -2].sum()
                        d_sum = sel_vec[:, 0].sum()

                        if util_sum > max_res and np.abs(d_sum - sum_demands[gr_num]) <= 0.2 * sum_demands[gr_num] :
                            max_res = util_sum
                            best_vec = sel_vec

                if best_vec.size > 0:
                    total_utility += max_res
                    selected_items.extend(best_vec)

        if final_total_utility < total_utility:
            final_total_utility = total_utility
            final_selected_items = selected_items

    return np.stack(final_selected_items), final_total_utility

In [ ]:
def median_quick_select(items, W):

    def subProcedure(remaining, new_W):
        if remaining == [] or min(i[0] for i in remaining) > new_W:
            return []

        # sample random item
        random_item = random.choice(remaining)
        print(random_item)

        # split input into larger and smaller subsets based on the item
        smaller = [i for i in remaining if i < random_item]
        not_smaller = [i for i in remaining if random_item <= i]

        # compute the total cost of all smaller items
        smaller_cost = sum(i[0] for i in smaller)

        if smaller_cost <= new_W:
            # include all smaller elements and recurse on the rest with a smaller W
            return smaller + subProcedure(not_smaller, new_W - smaller_cost)
        else:
            # recurse on the smaller subset with the same W
            return subProcedure(smaller, new_W)

    # convert a list of items (a, b, c) to  a list of the form ((a, 0), (b, 1), (c, 2)) for
    # consistent tie breaking in comparison operations
    augmented_items = [(j, i) for (i, j) in enumerate(items)]

    # compute the solution over the augmented items
    solution = subProcedure(augmented_items, W)

    # return the un-augmented items from the solution
    return [i[0] for i in solution]

In [ ]:
def ufp_quick_select(groups,y_pred):
    if type(y_pred) != np.ndarray:
        prediction = y_pred.cpu().detach().numpy()
    else:
        prediction = y_pred
    selection=[]
    result=0
    for i,group in enumerate(groups):
        if group.size>0:
            utilities=group[:,-2]
            print(utilities.max())
            U_constr=y_pred[i,2]
            print(U_constr)
            result+=sum(median_quick_select(utilities,U_constr))
    return result

In [ ]:
states=list(glob.glob('states/*.npz'))

In [ ]:
answer=list(glob.glob('answers/*.npy'))

In [ ]:
nums=len(states)

In [ ]:
def func(idx):
    state = np.load(states[idx])
    clients_list = state['cl']
    time_slot_capacity = state['tslot']
    quad_constr = state['quad_constr']
    f_groups, tot_dem, min_av_max_caps, av_cap_bytime, q_c, min_util, max_util, tot_util,util_type = problem_grouping(
        clients_list,
        time_slot_capacity,
        quad_constr)
    inp, lab = data_preprocess(f_groups, tot_dem, q_c, min_av_max_caps, av_cap_bytime,
                           min_util, max_util, tot_util,util_type) 
    ans=np.load(answer[idx])
    return clients_list,time_slot_capacity,f_groups,tot_dem,np.float32(inp),np.float32(lab),ans

In [ ]:
clients_list,time_slot,groups,total_demand,inp,lab,ans=func(100)
inp=torch.from_numpy(np.float32(inp[None,:])).to(device)
pred=knapsack_model(inp)
y_pred=pred.cpu().detach().numpy().squeeze()
utilities=np.concatenate([group[:,-2] for group in groups])
start=time.time()
result=ufp_quick_select(groups,y_pred)
print(time.time()-start)

In [ ]:
result

In [ ]:
clients_list,time_slot,groups,total_demand,inp,lab,ans=func(100)
inp=torch.from_numpy(np.float32(inp[None,:])).to(device)
pred=knapsack_model(inp)
y_pred=pred.cpu().detach().numpy().squeeze()
utilities=np.concatenate([group[:,-2] for group in groups])
start=time.time()
sel_it,util=selection_algorithm(clients_list,time_slot,groups,total_demand,y_pred)
print(time.time()-start)

In [ ]:
utilities[ans==1].sum()

In [ ]:
util

In [ ]:
sel_it.shape

In [ ]:
ans[ans==1].shape

In [ ]:
clients_list,time_slot,groups,total_demand,inp,lab,ans=func(100)
inp=torch.from_numpy(np.float32(inp[None,:])).to(device)
pred=knapsack_model(inp)
y_pred=pred.cpu().detach().numpy().squeeze()
start=time.time()
sel_it,util=selection_algorithm2(clients_list,time_slot,groups,total_demand,y_pred)
print(time.time()-start)

In [ ]:
util

In [ ]:
clients_list,time_slot,groups,total_demand,inp,lab,ans=func(100)
inp=torch.from_numpy(np.float32(inp[None,:])).to(device)
pred=knapsack_model(inp)
y_pred=pred.cpu().detach().numpy().squeeze()
start=time.time()
sel_it,util=selection_algorithm3(clients_list,time_slot,groups,total_demand,y_pred)
print(time.time()-start)